# Download
1. nltk

In [1]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


# Import

In [2]:
import json
import random
import re
import nltk
from nltk.stem import WordNetLemmatizer

## Download ntlk data packs

In [5]:
nltk.download("punkt", quiet=True)
nltk.download("wordnet", quiet=True)
nltk.download("omw-1.4", quiet=True)

lemmatizer = WordNetLemmatizer()

# Basic function

In [8]:
# pass a path and return a object
def load_json(path: str) -> object:
    try:
        with open(path, 'r', encoding="utf-8") as f:
            data = json.load(f)
        return data
    except:
        print(f"Error: {path} not found.")
        return None

In [16]:
# remove punctuation marks and lemmatized the passed user input
# then return the processed string
def preprocess_input(user_input: str) -> str:
    # preprocess
    lower_input = user_input.lower()
    clean_input = (r'[^\w\s]', '', lower_input)

    # tokenization
    tokens = nltk.word_tokenize(clean_input)

    # lemmatization
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    return "".join(lemmatized_tokens)

In [17]:
# pass a user input and the intent of chat bot
# and will return a string of reply
def get_response(user_input: str, intents_data: object) -> str:
    preprocessed_input = preprocess_input(user_input)

    for intent in intents_data["intents"]:
        for pattern in intent["patterns"]:
            match = re.search(pattern, user_input, re.IGNORECASE)
            vague_match = preprocess_input(pattern) in preprocessed_input


            if intent["tag"] == "provide_name" and match:
                capture_name(match)
            if match or vague_match:
                response = random.choice(intent["responses"])
                response = replace_with_memory(response)

                return response
    
    # default message
    return "I'm sorry. I didn't understand. Please say again."

# Advanced function

In [10]:
# Memory
# key -> content
memory = {}

In [18]:
# iterate the memory and try to replace the key name inside a curly brackets with its value
def replace_with_memory(response: str) -> str:
    for key in memory:
        if f"{{{key}}}" in response:
            response = response.replace(memory[key])

    return response

In [19]:
# pass a match info and a key name to store info into memory
def capture_information(match_data: re.Match, key: str) -> None:
    if match_data.groups():
        value = match.group(1)
        memory[key] = name

# Main loop

In [21]:
def chatbot_main() -> None:
    instents_data = load_json("./intents.json")
    if not instents_data:
        return

    print(instents_data["welcome_message"])
    print("==================================")
    while True:
        user_input = input("You: ")

        if user_input.lower() in ["bye", "quit"]:
            print(instents_data["exit_message"])
            break

        response = get_response(user_input, instents_data)
        print(f"Library Chatbot: {response}")

In [23]:
if __name__ == "__main__":
    chatbot_main()

Library Chatbot: Hello! I am your personal library assistant. What cna I help you.
Type 'bye' or 'quit' to exit.


You:  bye


Library Chatbot: Goodbye! Happy reading!
